In [1]:
from pyxll_notebook.server import xl_func

In [2]:
@xl_func
def test_func(a, b):
    """test func takes two args
    :param a: a
    :param b: b
    """
    return a + b

In [3]:
@xl_func("int, int: int[][]", auto_resize=True)
def auto_resize_test(rows, cols):
    result = []
    i = 0
    for r in range(rows):
        row = []
        for c in range(cols):
            row.append(i)
            i += 1
        result.append(row)
    return result

In [5]:
from pyxll_notebook.server import RTD
from datetime import datetime
import threading
import logging
import time


class CurrentTimeRTD(RTD):
    """CurrentTimeRTD periodically updates its value with the current
    date and time. Whenever the value is updated Excel is notified and
    when Excel refreshes the new value will be displayed.
    """

    def __init__(self, format):
        initial_value = datetime.now().strftime(format)
        super(CurrentTimeRTD, self).__init__(value=initial_value)
        self.__format = format
        self.__running = True
        self.__thread = threading.Thread(target=self.__thread_func)
        self.__thread.start()

    def connect(self):
        # Called when Excel connects to this RTD instance, which occurs
        # shortly after an Excel function has returned an RTD object.
        print("CurrentTimeRTD Connected")

    def disconnect(self):
        # Called when Excel no longer needs the RTD instance. This is
        # usually because there are no longer any cells that need it
        # or because Excel is shutting down.
        self.__running = False
        print("CurrentTimeRTD Disconnected")

    def __thread_func(self):
        while self.__running:
            # Setting 'value' on an RTD instance triggers an update in Excel
            new_value = datetime.now().strftime(self.__format)
            if self.value != new_value:
                self.value = new_value
            time.sleep(0.5)

In [6]:
@xl_func("string format: rtd")
def remote_rtd_current_time(format="%Y-%m-%d %H:%M:%S"):
    """Return the current time as 'real time data' that
    updates automatically.

    :param format: datetime format string
    """
    return CurrentTimeRTD(format)